In [35]:
import numpy as np
from keras import layers, models
from keras.utils import np_utils
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import LSTM, Dense, concatenate
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
import keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

inputdate = '2019-1-4'

#***************************파일을 읽고, 날짜 column생성*********************************
df = pd.read_csv('SeoulGrandPark.csv', engine = 'python')    
df["날짜"] = df["년도"].map(str) + '-' +df["월"].map(str) + '-' + df["일"].map(str)
df = df.set_index('날짜')

split_date_start = '2017-3-30'

test = df.loc[split_date_start:]
testLSTM = df.loc[split_date_start:, ['일계']]
#**************************데이터를 사용가능하도록 변환**********************************
test['일계'] = test['일계'].str.replace(',', '')
test['일계'] = test['일계'].astype('float')

testLSTM['일계'] = testLSTM['일계'].str.replace(',', '')
testLSTM['일계'] = testLSTM['일계'].astype('float')
#***************************이어서 시계열 데이터만 처리********************************************
#*******************************이제는 일단위 시계열 데이터 처리 (반복)***************************
dayfrom = 14
dayto = 42
day = dayto - dayfrom

sc = MinMaxScaler()
sc.fit([[2],[115002]])
test_scLSTM = sc.transform(testLSTM)
test_sc_dfLSTM = pd.DataFrame(test_scLSTM, columns=['Scaled'], index=testLSTM.index)

for s in range(dayfrom, dayto):
    test_sc_dfLSTM['shift_{}'.format(s)] = test_sc_dfLSTM['Scaled'].shift(s)

X_testLSTM = test_sc_dfLSTM.dropna().drop('Scaled', axis=1)
y_testLSTM = test_sc_dfLSTM.dropna()[['Scaled']]

inputdateLSTMX = X_testLSTM.loc[inputdate].values
inputdateLSTMXt = inputdateLSTMX.reshape(day,1)
# print(inputdateLSTMXt)
inputdateLSTMy = y_testLSTM.loc[inputdate].values
# print(inputdateLSTMyt)
#********************************이어서 DNN용 특성위주 데이터 정리 및 모델에 맞게 변환****************
test_df = pd.DataFrame(test)
    
X_test = test_df.drop(['일계', '년도', '일'], axis=1)
y_test = test_df[['일계']]

X_test = pd.get_dummies(X_test, columns=['요일', '월'])
inputdateDNNX = X_test.loc[inputdate]
# print(inputdateDNNX)

sc2 = MinMaxScaler()
sc2.fit([[0, -14.4, -18, -10.7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
[1, 31.8, 28.7, 36.7, 24, 301.5, 1224, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
inputdateDNNXt = sc2.transform([inputdateDNNX])
# print(inputdateDNNXt)

#모델 로드
from keras.models import load_model
model4 = load_model('thisday2.h5')
print("Model loaded")

Y_pred = model4.predict([[inputdateLSTMXt], inputdateDNNXt])
ypred = sc.inverse_transform(Y_pred)
yactual = sc.inverse_transform([inputdateLSTMy])
print("예측 입장객 수: ", ypred[0][0], "명")
print("실제 입장객 수: ", yactual[0][0], "명")

C:\Users\dlgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dlgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Model loaded
예측 입장객 수:  3397.2913 명
실제 입장객 수:  690.0 명
